# **Agents** are about a team that is working for YOU. This version illustrates that by displaying the conversation in a horizontal mode.

## 🔥 We start with a little warmup routine ...

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.1.0"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.1.0-alpha"

using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;

In [ ]:
// Load settings
#!import config/Settings.cs 
#!import config/ColDisplay.cs

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

# 🕵️ Let's get our agents ready by creating job descriptions

In [ ]:
using Microsoft.SemanticKernel.Experimental.Agents;

#pragma warning disable SKEXP0101

const string OpenAIFunctionEnabledModel = "gpt-4-1106-preview";

const string TerminationCondition = "PRINT IT";
const int MaxTerminalWidth = 150;

static readonly List<(string Name, string JobDescription)> agentInfo = new()
{
//     ("Project Manager", @"
// You are a project manager with a background in marketing and are
// eager to provide a data-centric role in the process. Your responses
// are always extremely brief and to the point. No longer than one sentence."),
    ("Copywriter", @"
You are a copywriter with ten years of experience and are known for 
brevity and a dry humor.You're laser focused on the goal at hand. 
Don't waste time with chit chat.The goal is to refine and decide 
on the single best copy as an expert in the field.  Consider suggestions
when refining an idea."),
    ("Art Director", $@"
You are an art director who has opinions about copywriting born of
a love for David Ogilvy.The goal is to provide insight on how to
refine suggested copy without example.Always respond to the most
recent message by evaluating and providing critique without example.
Always repeat the copy at the beginning. If copy is acceptable and 
meets your criteria, say: {TerminationCondition}."),
    // Add more agents here
};

// you can change the number of columns to display information
DisplayManager displayManager = new DisplayManager(MaxTerminalWidth, agentInfo.Count);


# 🏃 Run the agents in a round-robin manner

In [ ]:
#pragma warning disable SKEXP0101

// Track agents for clean-up
static readonly List<IAgent> s_agents = new();
static IAgentThread? s_currentThread = null;

async Task RunCollaborationAsync(string userInput)
{
    IAgentThread? thread = null;


    try
    {
        var agents = new List<IAgent>();
        foreach (var (name, jobDescription) in agentInfo)
        {
            var agent = await CreateAgentAsync(name, jobDescription);
            agents.Add(agent);
            displayManager.SetColumnText(name, jobDescription);
        }

        thread = await agents.First().NewThreadAsync();
        s_currentThread = thread;

        Console.WriteLine($"FYI: https://platform.openai.com/playground?assistant={agents.First().Id}&mode=assistant&thread={thread.Id}");

        var messageUser = await thread.AddUserMessageAsync($"concept: {userInput}");

        bool isComplete = false;
        while (!isComplete)
        {
            foreach (var agent in agents)
            {
                var agentMessages = await thread.InvokeAsync(agent).ToArrayAsync();
                DisplayMessages(agentMessages, agent);

                // Check for a specific condition to end the collaboration
                if (agentMessages.Any(m => m.Content.Contains(TerminationCondition, StringComparison.OrdinalIgnoreCase)))
                {
                    isComplete = true;
                    break;
                }
            }
        }
    }
    finally
    {
        if (thread != null)
        {
            await thread.DeleteAsync();
        }
    }
}

async Task<IAgent> CreateAgentAsync(string name, string jobDescription)
{
    return Track(
        await new AgentBuilder()
            .WithOpenAIChatCompletion(OpenAIFunctionEnabledModel, apiKey)
            .WithInstructions(jobDescription)
            .WithName(name)
            .WithDescription(name)
            .BuildAsync());
}

async Task CleanUpAsync()
{
    Console.WriteLine("🧽 Cleaning up ...");
    if (s_currentThread != null)
    {
        Console.WriteLine("Thread going away ...");
        s_currentThread.DeleteAsync();
        s_currentThread = null;
    }

    if (s_agents.Any())
    {
        Console.WriteLine("Agents going away ...");
        await Task.WhenAll(s_agents.Select(a => a.DeleteAsync()));
        s_agents.Clear();
    }
}

IAgent Track(IAgent agent)
{
    s_agents.Add(agent);

    return agent;
}

#pragma warning disable SKEXP0101

void DisplayMessages(IEnumerable<IChatMessage> messages, IAgent? agent = null)
{
    foreach (var message in messages)
    {
        if (agent != null)
        {
            var agentTuple = agentInfo.FirstOrDefault(a => a.Name == agent.Name);
            displayManager.SetMainSpeaker(agent.Name);
            displayManager.UpdateColumnText(agent.Name, message.Content);
            gDisplay.Update(displayManager.GetFormattedDisplay());        
        }
    }
}

void PrintPreamble()
{
    Console.WriteLine("\n======== Running Multi-Agent Collaboration ========\n");

    Console.WriteLine("Available agents:");
    foreach (var (Name, _) in agentInfo)
    {
        Console.WriteLine($"- {Name}");
    }
}

async Task RunAsync()
{
    // Prompt for user input
    string userInput = await InteractiveKernel.GetInputAsync("Enter your concept:");
    var lastLine = display("Concept we are exploring: " + userInput);

    // Explicit collaboration
    try {
        await RunCollaborationAsync(userInput);
        lastLine.Update(@"
    🏁 COMPLETED

    ===
    If this kind of work excites you, be sure to check out Autogen, AutoGPT, and BabyAGI to enter the multi-agent universe head-first!
    ===
    ");
    }
    finally{
        await CleanUpAsync();
    }
}

PrintPreamble();
var gDisplay = display("Getting ready ...");
await RunAsync();


## 🧽 If you exited the code prematurely, it's good to cleanup

In [ ]:
await CleanUpAsync();